# Authors:

- Luca Erbì
- Gabriele Lorenzo


# Lab 3: Fewshot ICL

As knowledge graph requires background in SPARQL and/or LLM finetuning, this lab won't be totally related to what you saw in today's course.

We'll be delving into In Context Learning (ICL), in particular ICL fewshot, and trying to understand how it works and when to use it. To do this, we'll be using the Transformer Library, a Mistral LLM and an emotion classification dataset.

The laboratory is divided into 4 sections: 0. Setup: This section is dedicated to installing modules, loading models and loading data.You don't need to code, just run it.

1. Zeroshot Classification: Some of you may have had trouble finding a prompt that always returned a “well-formed” answer in the last lab. In this section, we'll use a “well-formed” prompt to perform zeroshot classification.
2. Fewshot Classification - Random Retrieval: One of the most common methods of improving ICL classification is to add demonstrations to the prompt. This helps the LLM to “properly format” the response and can also give semantic information about how to solve the task. In this section, we will use random retrieved demonstration and compare the results with those of section 1.
3. Fewshot Classification - Vector-based Retrieval: Extracting random demonstrations in fewshot classification can introduce bias. In addition, most semantically relevant demonstrations are not taken into account. As we did with RAG, we will use a vector representation of the example to retrieve the most relevant demonstrations.
4. Constrained Decoding: Finally, we'll discovering the `outlines` library, which contains modules that are useful to do constrained decoding.

At the end of each section (except section 0.), there's a question to answer.


## 0. Setup


In [1]:
!pip install transformers bitsandbytes accelerate datasets outlines scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.7/912.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found e

In [2]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

import torch

# Put your hugging face token here: https://huggingface.co/docs/hub/en/security-tokens
# You need to fill the access form with your huggingface account on this link: https://huggingface.co/mistralai/Ministral-8B-Instruct-2410
hf_token = ""
llm_name = "mistralai/Ministral-8B-Instruct-2410"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left", token=hf_token)
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
    token=hf_token
)
# Set LLM on eval mode.
llm.eval()


tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 4096)
    (layers): ModuleList(
      (0-35): 36 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSN

In [3]:
# Set up our generation configuration.
# We set max_new_token to 128 to reduce computation time (we may also lose some accuracy).
# We disable beamsearch to ensure reproducibility (we may lose some accuracy).
generation_config = GenerationConfig(
  max_new_tokens = 128,
  do_sample=False,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

In [4]:
from datasets import load_dataset
import random
random.seed(42)

id2label = {0:"sadness", 1:"joy", 2:"love", 3:"anger", 4:"fear", 5:"surprise"}

# Dataset: https://huggingface.co/datasets/dair-ai/emotion
ds = load_dataset("dair-ai/emotion", "split")
examples = [{"text":ex["text"], "label":id2label[ex["label"]]} for ex in ds['test'].to_list()]
random.shuffle(examples)

# Split examples and keep only a few samples to have short computation time.
test, train = examples[:100], examples[100:500]
print(f"Train len {len(train)}. Test len {len(test)}")
print(f"First example of test:\n{test[0]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Train len 400. Test len 100
First example of test:
{'text': 'i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history', 'label': 'surprise'}


## 1. Zero-shot Classification

It's very similar to what you've done last time, so we're providing you with most of the code. The only thing you need to code yourself is the parse_answer function.

- We adapted the recommended classification prompt from: https://docs.mistral.ai/guides/prompting_capabilities/
- The purpose of this function is to return the first occurrence of a correct label (sadness, joy, love, anger, fear, surprise)
- We want to return "" if no answer is found.
- You can use regex or string functions.

There is a cell below to test your code. The output should be:

```
##### Example 0 #####
# You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the category. Do not include the word "Category". Do not provide explanations or notes.

<<<
Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label:
>>>
# sadness
# sadness
```


In [5]:
import re

zeroshot_prompt = """
You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

<<<
Sentence: {sentence}
Label:
>>>
""".strip()


def generate(prompt, llm=llm, generation_config=generation_config):

  # Create turns with the given prompt
  turns = [
    {'role':'user', 'content':prompt}
  ]

  # Tokenize turns.
  input_ids = tokenizer.apply_chat_template(turns, return_tensors='pt').to('cuda')

  # Ensure we don't use gradient to save memory space and computation time.
  with torch.no_grad():
    outputs = llm.generate(
      input_ids,
      generation_config
    )

  # Recover and decode answer.
  answer_tokens = outputs[0, input_ids.shape[1]:-1]
  return tokenizer.decode(answer_tokens).strip()


def parse_answer(answer):
    return answer.strip()

In [6]:
# Test your code
example = test[0]

prompt = zeroshot_prompt.format(sentence=example["text"])
answer = generate(prompt)
prediction = parse_answer(answer)

print("##### Example 0 #####")
print(f"# {prompt}")
print(f"# {answer}")
print(f"# {prediction}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


##### Example 0 #####
# You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

<<<
Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label:
>>>
# sadness
# sadness


Now apply the zero-shot prompt on the full test dataset. You need report:

- Accuracy (recall: number of correct answers divided by number of samples)
- Ratio of missing answer (i.e "." answer)

It should take 3 to 5 minutes to run.


In [7]:
from tqdm.notebook import tqdm

missing_answer = 0
correct_answer = 0

for example in tqdm(test):  # tqdm allow you to track the progression of your loop.
    prompt = zeroshot_prompt.format(sentence=example["text"])
    answer = generate(prompt)
    prediction = parse_answer(answer)

    if prediction == example["label"]:
        correct_answer += 1
    elif prediction == "":
        missing_answer += 1

print(f"\nAccuracy: {correct_answer / len(test)} ({correct_answer}/{len(test)})")
print(f"Missing answer: {missing_answer / len(test)} ({missing_answer/len(test)})")

  0%|          | 0/100 [00:00<?, ?it/s]


Accuracy: 0.63 (63/100)
Missing answer: 0.0 (0.0)


Note: We always find an answer, because we've used a “well-formed” prompt and because Mistral is good at following this type of instruction. If you try with the Lama-3, some answers may be missing.

**Question: Are we sure that all these answer are "well-formed" answer ?**


While the model is good at following the prompt and outputting one of the specified labels, it doesn't guarantee that all answers will be semantically correct or aligned with the actual sentiment of the input text. The model can still generate a valid label even if it's aware about an actual answer.


## 2. Fewshot Classification - Random Retrieval:

Now we have a working zeroshot solution. Our next next step is to use demonstrations. We will start be implementing a random few shot generation. You need to implement 3 functions:

- format_demo, wich format a given example into a demonstration string
- format_demos, wich format a given list of example into a demonstration string (try to use format_demo)
- get_random_demo, wich return k random examples. (you should use random.choice. https://docs.python.org/3/library/random.html)

There is a cell below to test your code. The output should be:

```
##### format_demo #####
# Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label: surprise.


##### format_demos #####
# Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label: surprise.

Sentence: im feeling optimistic to finish out these last two weeks strong and probably continue with what i have been doing
Label: joy.

Sentence: i feel complacent and satisfied
Label: joy.

Sentence: im the only one with all the feelings and emotions and thats just pathetic of me to do so
Label: sadness.

Sentence: i just sat there in my group feeling really depressed because my book just had to go missing at this time
Label: sadness.


##### Example 0 #####
# You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

####
Here are some examples:

Sentence: i feel inspired so many thing i want to write down
Label: joy.

Sentence: i feel like i should have some sort of rockstar razzle dazzle lifestyle but i would at least like to spend a third of my life doing something i feel is worthwhile
Label: joy.

Sentence: i continue to write this i feel more and more distraught
Label: fear.

Sentence: i feel that third situation pretty much sums up my feelings toward this title
Label: joy.

Sentence: i remember wanting to fit in so bad and feeling like no one liked me
Label: love.
####

<<<
Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label:
>>>
# sadness
# sadness
```


In [8]:
fewshot_prompt = """
You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

####
Here are some examples:

{examples}
####

<<<
Sentence: {sentence}
Label:
>>>
""".strip()

def format_demo(demo):
    return f"Sentence: {demo['text']}\nLabel: {demo['label']}"

def format_demos(demos):
    return "\n\n".join(format_demo(demo) for demo in demos)

def get_random_demo(k, train=train):
    return random.choices(train, k=k)

In [9]:
# Test your code !
print("##### format_demo #####")
print(f"# {format_demo(test[0])}")

print("\n\n##### format_demos #####")
print(f"# {format_demos(test[:5])}")

random.seed(42)

example = test[0]
demos = format_demos(get_random_demo(5))

prompt = fewshot_prompt.format(examples=demos, sentence=example["text"])
answer = generate(prompt)
prediction = parse_answer(answer)

print("\n\n##### Example 0 #####")

print(f"# {prompt}")
print(f"# {answer}")
print(f"# {prediction}")

##### format_demo #####
# Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label: surprise


##### format_demos #####
# Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label: surprise

Sentence: im feeling optimistic to finish out these last two weeks strong and probably continue with what i have been doing
Label: joy

Sentence: i feel complacent and satisfied
Label: joy

Sentence: im the only one with all the feelings and emotions and thats just pathetic of me to do so
Label: sadness

Sentence: i just sat there in my group feeling really depressed because my book just had to go missing at this time
Label: sadness


##### Example 0 #####
# You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined

Now apply the fewshot prompt on the full test dataset. You need report:

- Accuracy (recall: number of correct answers divided by number of samples)
- Report them for k=1 and k=5

It should take 5 to 7 minutes to run.


In [10]:
from tqdm.notebook import tqdm

random.seed(42)

correct_answers = []
missing_answers = []

for k in tqdm([1, 5]):
    missing_answer = 0
    correct_answer = 0

    for example in tqdm(test):  # tqdm allow you to track the progression of your loop.
        demos = format_demos(get_random_demo(k))
        prompt = fewshot_prompt.format(examples=demos, sentence=example["text"])
        answer = generate(prompt)
        prediction = parse_answer(answer)

        if prediction == example["label"]:
            correct_answer += 1
        elif prediction == "":
            missing_answer += 1

    correct_answers.append(correct_answer)
    missing_answers.append(missing_answer)

print(f"Accuracy for k=1:\t{correct_answers[0] / len(test)} ({correct_answers[0]}/{len(test)})")
print(f"Missing answer for k=1:\t{missing_answers[0] / len(test)} ({missing_answers[0]/len(test)})")

print(f"Accuracy for k=5:\t{correct_answers[1] / len(test)} ({correct_answers[1]}/{len(test)})")
print(f"Missing answer for k=5:\t{missing_answers[1] / len(test)} ({missing_answers[1]/len(test)})")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy for k=1:	0.61 (61/100)
Missing answer for k=1:	0.0 (0.0)
Accuracy for k=5:	0.67 (67/100)
Missing answer for k=5:	0.0 (0.0)


**Question: What are the limits of using a single demonstration? What are the limits of using too many demonstrations?**

With only one demonstration, the model may not be able to understand the task. With too many demonstrations, the model may be confused by the amount of information (we introduce noise).


## 3. Fewshot Classification - Vector-based Retrieval

Now, we want to improve demonstration by the vector representation of our sentence. This is close to what we did when we used RAG on wikipedia page. But here, we'll do it manually and step by step.

To do so, we need to calculate the vector representation of our training dataset. To do this, we'll code a function that returns a vector for a given example. We'll use our LLM hidden states to do this. It's not optimal, but we won't have to load another model.

First, look at the mistral architecture:

```
MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 4096)
    (layers): ModuleList(
      (0-35): 36 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
  )
  (lm_head): Linear(in_features=4096, out_features=131072, bias=False)
)
```

There are 36 transformer layers and 1 language model (LM) layer. Each layer will take the following shape: [1, N_TOKENS, N_PARAMS]. We want to extract the vector of the last token from the last transformer. To do so:

- Encode the sentence without any template. `tokenizer.encode(...)`
- Use the `output_hidden_states` keyword of the llm forward function.
- Select the last transformer layer (be careful, don't take the LM layer).
- Select the last token.
- Convert the vector to numpy `.to('cpu').float().numpy()` and return it.

There is a cell below to test your code. The output should be:

```
# (4096,)
# [ 4.59375    -9.          0.80078125 ...  0.890625   -0.20019531
 -0.62109375]
```


In [11]:
def get_hidden_repr(text, llm=llm, prompt_template=zeroshot_prompt):
    input_ids = tokenizer.encode(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = llm(input_ids, output_hidden_states=True).hidden_states[-2]

    return outputs[0, -1, :].to("cpu").float().numpy()

In [12]:
example = train[0]
vector = get_hidden_repr(example["text"])
print("#", vector.shape)
print("#", vector)

# (4096,)
# [ 4.59375    -9.          0.80078125 ...  0.890625   -0.20019531
 -0.62109375]


Now, we need to get the hidden represation vector for all examples in the train and the test datasets.

You should store the vector directly in the example dict: `example["vector"] = ...`

Both should take 3 - 5 mins to run.


In [13]:
for example in tqdm(train): # tqdm allow you to track the progression of your loop.
    example["vector"] = get_hidden_repr(example["text"])

  0%|          | 0/400 [00:00<?, ?it/s]

In [14]:
# Same for test examples !
for example in tqdm(test): # tqdm allow you to track the progression of your loop.
    example["vector"] = get_hidden_repr(example["text"])

  0%|          | 0/100 [00:00<?, ?it/s]

Now that we have our vector representations. We want a function that compute the cosine similarity between 2 examples.

- Use the function from sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
- Be careful, you have to reshape each vector to: [1, 4096]

There is a cell below to test your code. The output should be:

```
# a . a = 1.0000019073486328
# a . b = 0.930396318435669
```


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(example_a, example_b):
    vector_a = example_a["vector"].reshape(1, -1)
    vector_b = example_b["vector"].reshape(1, -1)

    return cosine_similarity(vector_a, vector_b)[0][0]

In [16]:
# Test your code !
a, b = train[0], train[1]

print(f"# a . a = {compute_similarity(a, a)}")
print(f"# a . b = {compute_similarity(a, b)}")

# a . a = 1.0000028610229492
# a . b = 0.5770441293716431


Last step, we want a function that retrieve the k more similar demonstrations of the train examples given a test example.

There is a cell below to test your code. The output should be:

```
# surprise - i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
#  joy - i feel lucky that theyve chosen to share their lives with me

joy - i feel our world then was a much more innocent place

joy - i know he does the same thing for so many passersby i feel special truly welcome in his country

joy - i do know that i tell some people if i feel that their question is sincere some of my sacred treasures

anger - i feel appalled that i took advantage of my old friend s kindness

```


In [17]:
def get_k_similar_demo(example, k, train=train):
    similarities = [
        (compute_similarity(example, other), other) for other in train
    ]
    similarities.sort(reverse=True, key=lambda x: x[0])

    return [example for _, example in similarities[:k]]

In [18]:
# Test your code !
example = test[0]
print(f"# {example['label']} - {example['text']}")
print("# ", "\n\n".join([f"{ex['label']} - {ex['text']}" for ex in get_k_similar_demo(example, 5)]))

# surprise - i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
#  joy - i feel lucky that theyve chosen to share their lives with me

joy - i feel our world then was a much more innocent place

joy - i know he does the same thing for so many passersby i feel special truly welcome in his country

joy - i do know that i tell some people if i feel that their question is sincere some of my sacred treasures

anger - i feel appalled that i took advantage of my old friend s kindness


Now apply the fewshot prompt on the full test dataset. You need report:

- Accuracy (recall: number of correct answers divided by number of samples)
- Report them for k=1 and k=5

It should take 5 to 7 minutes to run.

Your results should be:

```
##### k=1 #####
Accuracy:  0.65
##### k=5 #####
Accuracy:  0.63
```


In [19]:
from tqdm.notebook import tqdm

correct_answers = []
missing_answers = []

for k in tqdm([1, 5]):
    missing_answer = 0
    correct_answer = 0

    for example in tqdm(test):  # tqdm allow you to track the progression of your loop.
        demos = format_demos(get_k_similar_demo(example, k))
        prompt = fewshot_prompt.format(examples=demos, sentence=example["text"])
        answer = generate(prompt)
        prediction = parse_answer(answer)

        if prediction == example["label"]:
            correct_answer += 1
        elif prediction == "":
            missing_answer += 1

    correct_answers.append(correct_answer)
    missing_answers.append(missing_answer)

print(f"##### k = 1 #####")
print(f"Accuracy: {correct_answers[0] / len(test)}")

print("##### k = 5 #####")
print(f"Accuracy: {correct_answers[1] / len(test)}")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

##### k = 1 #####
Accuracy: 0.65
##### k = 5 #####
Accuracy: 0.6


**Question: What could be the main issue with this approach? How can it be mitigated?**


The main problem could be that semantic similarity (what vectors capture) doesn't perfectly equal task relevance (what's best for in-context learning). Vector embeddings might group sentences by general topic but miss nuances crucial for a specific classification task. A solution could be using task-Specific Embeddings and/or an hybrid retrieval method.


## 4. Constrained Decoding

Last exercise, we will use the `outlines` package to do constrained generation. This main idea is to guide the generation of the LLM to get the good output formats.

We will use the choices module. Here is the documentation: https://dottxt-ai.github.io/outlines/latest/reference/generation/choices/

There is an example below on how to use it on 1 example. We let you apply this methods to the test dataset. You need report:

- Accuracy (recall: number of correct answers divided by number of samples)
- Ratio of missing answer (i.e "E." answer)
- Report them for k=1 and k=5

It should take 3 to 5 minutes to run.

Your results should be:

```
Accuracy:  0.38
```


In [20]:
from outlines import models, generate
from tqdm.notebook import tqdm

random.seed(42)

llm_model = models.Transformers(llm, tokenizer)

def constrained_generation(sentence):
    options = ["sadness", "joy", "love", "anger", "fear", "surprise", "E."]
    generator = generate.choice(llm_model, options)
    answer = generator(zeroshot_prompt.format(sentence=sentence))
    return answer


missing_answer = 0
correct_answer = 0

for example in tqdm(test):  # tqdm allow you to track the progression of your loop.
    prediction = constrained_generation(sentence = example["text"])

    # Check correctness
    if prediction == example["label"]:
        correct_answer += 1

    # Check for missing answers
    if prediction == "E." or prediction == "":
        missing_answer += 1

# Report results
print("\n##### Results #####")
print(f"Accuracy: {correct_answer / len(test)}")
print(f"Missing Ratio: {missing_answer / len(test)}")


  0%|          | 0/100 [00:00<?, ?it/s]


##### Results #####
Accuracy: 0.35
Missing Ratio: 0.0


**Question: Now that you've used all these solutions, when should you use zeroshot? when should you use fewshot? when should you use constrained decoding?**


Zero-shot sould be used for rapid, straightforward tasks when training data is scarce. While few-shot should be used when you have example data and require improved accuracy.
Constrained decoding when you need to guarantee a specific output structure, that could be used for generation task for example, where we need a data type in a specific format.


## Bonus

Try to use differents modules of `outlines` like json, pydantic or regex ...

Compare this results with previous ones !


In [21]:
from enum import Enum
from pydantic import BaseModel, Field
from outlines import models, generate
from tqdm.notebook import tqdm

random.seed(42)

llm_model = models.Transformers(llm, tokenizer)

class EmotionEnum(str, Enum):
  sadness = "sadness"
  joy = "joy"
  love = "love"
  anger = "anger"
  fear = "fear"
  surprise = "surprise"

class Emotion(BaseModel, use_enum_values=True ):
    emotion: str=EmotionEnum

def constrained_generation(sentence):
    generator = generate.json(llm_model, Emotion, whitespace_pattern=r"[\n\t ]*")

    answer = generator(zeroshot_prompt.format(sentence=sentence))

    return answer

missing_answer = 0
correct_answer = 0

for example in tqdm(test):  # tqdm allow you to track the progression of your loop.
    prediction = constrained_generation(sentence = example["text"])

    # Check correctness
    if prediction.emotion == example["label"]:
        correct_answer += 1

    # Check for missing answers
    if prediction.emotion not in EmotionEnum.__members__:
        missing_answer += 1

# Report results
print("\n##### Results #####")
print(f"Accuracy: {correct_answer / len(test)}")
print(f"Missing Ratio: {missing_answer / len(test)}")

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value <enum 'EmotionEnum'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)



##### Results #####
Accuracy: 0.38
Missing Ratio: 0.11


In [22]:
from outlines import models, generate
from tqdm.notebook import tqdm

random.seed(42)

llm_model = models.Transformers(llm, tokenizer)

def constrained_generation(sentence):
    generator = generate.regex(
        llm_model,
        r"sadness|joy|love|anger|fear|surprise|E\."
    )
    answer = generator(zeroshot_prompt.format(sentence=sentence))
    return answer

missing_answer = 0
correct_answer = 0

for example in tqdm(test):  # tqdm allow you to track the progression of your loop.
    prediction = constrained_generation(sentence = example["text"])

    # Check correctness
    if prediction == example["label"]:
        correct_answer += 1

    # Check for missing answers
    if prediction == "E." or prediction == "":
        missing_answer += 1

# Report results
print("\n##### Results #####")
print(f"Accuracy: {correct_answer / len(test)}")
print(f"Missing Ratio: {missing_answer / len(test)}")

  0%|          | 0/100 [00:00<?, ?it/s]


##### Results #####
Accuracy: 0.37
Missing Ratio: 0.0


The other solutions offer compelling approaches, particularly the one utilizing Pydantic. This method enables users to enforce output structure through type constraints, coupled with JSON formatting control.
In the end, the results are a bit random and could be better with fewshot prompting.
